In [ ]:
include("setup.jl")

In [ ]:
using ExcelReaders

In [ ]:
workspace = pwd()
f = openxl(string(workspace,"/tsp/roadtrip_quebec.xls"))

In [ ]:
data_cities = readxl(f, "Donnees!A9:A17")

In [ ]:
c = map(strip, data_cities)

In [ ]:
data_distances = readxl(f, "Donnees!C9:K17")

In [ ]:
nb_row, nb_col = size(data)
distances = zeros(Int, nb_row, nb_col)
for i = 1:nb_row
    for j = 1:nb_col
        distances[i,j] = Int(data_distances[i,j])
    end
end
distances

In [ ]:
using JLD

In [ ]:
save(string(workspace,"/tsp/distances.jld"),"dist",distances)
save(string(workspace,"/tsp/cities.jld"),"cities",c)

In [ ]:
d = load(string(workspace,"/tsp/distances.jld"),"dist")
c = load(string(workspace,"/tsp/cities.jld"),"cities")

In [ ]:
using JuMP

In [ ]:
N = 9
m = Model()

In [ ]:
@variable(m, x[1:N,1:N], Bin)

In [ ]:
for k=1:N
    @constraint(m,x[k,k] == 0)
end

In [ ]:
for i = 1:N
    for j = 1:N
        @constraint(m, sum(x[i,j] for i = 1:N) == 1)
        @constraint(m, sum(x[i,j] for j = 1:N) == 1)
    end
end

In [ ]:
function subsets(x::Vector{T}) where T
    res = Vector{T}[[]]
    for elem in x, j in eachindex(res)
        push!(res, [res[j] ; elem])
    end
    return res
end

In [ ]:
soustour = subsets([1:N;])

In [ ]:
nb_tour = 0
for t in soustour
    T = length(t)
    if 2 ≤ T ≤ N-1
        nb_tour += 2
        @constraint(m, sum(x[t[k],t[k+1]] for k=1:T-1) + x[t[T],t[1]] ≤ T-1)
        @constraint(m, sum(x[t[k+1],t[k]] for k=1:T-1) + x[t[1],t[T]] ≤ T-1)
    end
end
nb_tour

In [ ]:
sum(2^i for i = 2:8)

In [ ]:
@objective(m, Min, sum(d[i,j] * x[i,j] for i=1:N, j=1:N))

In [ ]:
m

In [ ]:
using Cbc

In [ ]:
setsolver(m, CbcSolver())

In [ ]:
status = solve(m)

In [ ]:
sol = getvalue(x)

In [ ]:
cities_visited = 1
city = 1
println(c[city])
while cities_visited ≠ N
    city = findfirst(sol[city,:] .== 1.0) 
    cities_visited += 1
    println(c[city])
end